## Anomaly(Abnormal) Detection Part2.

### 목차
DL 기반 모델링 (Autoencoder)
   1. 딥러닝 및 오토인코더 개념 설명
   2. 오토인코더 기본 모델링 (코딩)
   3. 오토인코더 동적 모델링 (코딩)
   4. 오토인코더 모델링의 비즈니스 평가 (코딩)

### 0. 들어가기전에 (딥러닝 간단 이해)

역전파 알고리즘은 신경망이 학습을 할 수 있게 하는 중요한 메커니즘입니다. 이 알고리즘은 네트워크의 가중치를 효율적으로 조정하기 위해 손실 함수(loss function)의 그래디언트(gradient)를 계산하는데 사용됩니다. 여기서 간단한 예와 함께 역전파의 기본 개념과 공식을 설명하겠습니다.

#### 딥러닝 특징과 시각적 이해

1. **딥러닝 특징**

- 딥러닝은 뉴런을 본딴 인공신경망으로 학습
- 딥러닝은 특징 추출 과정이 없음.
- 시각적 이해 : https://alexlenail.me/NN-SVG/index.html
  <br/>

1. **학습원리 (역전파 알고리즘)**
   <br/>
   <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fbu7rnO%2FbtsAm0em8x0%2FOqPL77F3nkKMzgPccksBe0%2Fimg.png" width="500">
   <br/>
   <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FPQ7bM%2FbtsAp1pFp0G%2Fpk7VO1EKTSUyc8wlZZxuh0%2Fimg.png" width="400">
   <br/>
   <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FWKpwj%2FbtsArne7UGR%2F5KPmuPGGSavivvK4vBtbUk%2Fimg.png" width="300">
   <br/>
   <img src="https://img-api.cboard.net/img.php?url=https://search.pstatic.net/common/?src=http:%2F%2Fblogfiles.naver.net%2FMjAxNzA2MjVfMTY2%2FMDAxNDk4MzE5NjkwOTM4.z7484d7GaYmp7T_fM8DCXz-KBqLGN5Yn6BsKwD8RleQg.nXMs_0IAr7Dc0JC4UvU3SHislj2R7IjM1kQAbsjAx-Ug.PNG.samsjang%2F%25C4%25B8%25C3%25B3.PNG&type=sc960_832" width="500">

#### 역전파의 기본 원리

역전파는 두 가지 주요 단계로 이루어집니다:

1. **순전파(Forward Propagation)**: 입력 데이터가 네트워크를 통과하며 각 층의 뉴런을 활성화시키고, 최종적으로 출력값을 생성.
2. **역전파(Backward Propagation)**: 출력값과 실제 타겟값과의 차이를 계산하여 손실을 도출하고, 이 손실을 네트워크를 거슬러 올라가면서 각 층의 가중치에 대한 그래디언트를 계산하여 가중치를 업데이트함.

#### 수학적 이해

- 신경망의 각 뉴런에서, 입력 \( x \)에 대한 가중치 \( w \)와 편향 \( b \)가 적용되고, 활성화 함수 \( f \)를 통해 출력 \( y \)가 결정됨. 예를 들어, 한 뉴런의 출력을 \( y = f(w \cdot x + b) \)로 모델링 할 수 있습니다. 여기서 \( f \)는 비선형 활성화 함수, 예를 들어 시그모이드나 렐루 함수가 될수 있음.

- ##### 그래디언트 계산

  손실 함수 \( L \)를 최소화하기 위해 필요한 것은 \( w \)와 \( b \)에 대한 \( L \)의 미분값, 즉 그래디언트를 계산. 그래디언트는 연쇄법칙(chain rule)을 사용하여 다음과 같이 계산 가능.

  \[ \frac{\partial L}{\partial w} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial w} \]
  \[ \frac{\partial L}{\partial b} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial b} \]

  여기서 \( \frac{\partial y}{\partial w} \)와 \( \frac{\partial y}{\partial b} \)는 각각 \( x \)와 \( 1 \)이므로,

  \[ \frac{\partial L}{\partial w} = \frac{\partial L}{\partial y} \cdot x \]
  \[ \frac{\partial L}{\partial b} = \frac{\partial L}{\partial y} \]

- ##### 가중치 업데이트

  가중치는 다음과 같이 업데이트 진행

  \[ w = w - \eta \cdot \frac{\partial L}{\partial w} \]
  \[ b = b - \eta \cdot \frac{\partial L}{\partial b} \]

  여기서 \( \eta \)는 학습률(learning rate) 를 의미.

  이러한 과정을 모든 층에 대해 반복하며 네트워크의 가중치를 업데이트 수행. 이것이 바로 딥러닝에서의 학습 과정이며, 역전파 알고리즘이 핵심적인 역할을 함.



### 1. AutoEncoder

- 개념 : Autoencoder는 신경망을 이용하여 입력 데이터의 압축된 표현을 학습하는 딥러닝 모델임. 기본적으로, autoencoder는 입력 데이터를 잘 복원할 수 있는 방식으로 데이터의 특징을 압축하고 재구성하는 과정을 통해 작동함. 이 모델은 주로 데이터의 차원 축소, 특징 추출, 또는 노이즈 제거 등에 사용됨.
- 구조
  1. 인코더(Encoder): 입력 데이터를 받아 내부의 더 낮은 차원으로 표현하는 latent space(잠재 공간)을 생성함. 이 과정에서 데이터의 중요한 특징이 추출되며, 데이터의 차원이 축소됨.
  2. 잠재 공간(Latent Space): 인코더에 의해 생성된, 입력 데이터의 압축된 표현임. 이 공간의 각 차원은 입력 데이터의 중요한 특징을 나타내며, 이를 통해 데이터의 핵심적인 구조를 파악할 수 있음.
  3. 디코더(Decoder): 잠재 공간의 표현을 다시 원래의 데이터 공간으로 매핑하여, 원본 데이터와 유사한 데이터를 재구성함.
- 학습순서
  1. 압축(인코딩) : 인코더를 통해 입력 데이터를 잠재 공간으로 압축함.
  2. 복원(디코딩) :잠재 공간의 표현을 디코더를 통해 다시 원래 차원으로 확장하여 데이터를 재구성함.
  3. 손실계산 및 학습 : 재구성된 데이터와 원본 데이터 간의 차이(손실)를 계산함. 이 손실을 최소화하는 방향으로 모델의 파라미터를 조정함. 이 손실은 일반적으로 Mean Squared Error(MSE) 등을 사용하여 계산함.
- 용도
  1. 이미지 데이터의 노이즈 제거
  2. 얼굴 인식 시스템에서의 특징 학습
  3. 추천 시스템에서의 사용자 행동 패턴 학습
  4. **이상 탐지**
- 이상탐지를 위한 AutoEncoder
  - 학습시에는 정상 데이터 포인트만 학습.
  - 검증셋과 테스트셋은 정상과 비정상 데이터 포인트롤 모두 포함함.
  - MSE로 복원(Reconstruction) Error 계산
  - Reconstruction Error 가 Threshold 이하면 정상, 그렇지 않으면 비정상으로 판단
  - Threshold를 정하는 기준


##### 라이브러리 로드

In [146]:
# import packages
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Reshape
from keras.losses import MeanSquaredError
from keras.backend import clear_session



In [3]:
import plotly.express as px
import plotly.graph_objects as go


In [121]:
import warnings
warnings.filterwarnings('ignore')


### 1) 오토인코더 기본 모델링 ( 세콤 데이터 이상 탐지 )

##### 데이터셋1(세콤데이터) 로드

In [305]:
data = pd.read_csv('dataset/secom.csv')
print(data.head())
print(data.shape)

   defeat             datetime    v021    v087    v088       v089    v114  \
0  normal  19/07/2008 11:55:00  1.4026  2.3895  0.9690  1747.6049  0.9460   
1  normal  19/07/2008 12:32:00  1.3825  2.3754  0.9894  1931.6464  0.9425   
2  defeat  19/07/2008 13:17:00  1.4123  2.4532  0.9880  1685.8514  0.9231   
3  normal  19/07/2008 14:43:00  1.4011  2.4004  0.9904  1752.0968  0.9564   
4  normal  19/07/2008 15:22:00  1.3888  2.4530  0.9902  1828.3846  0.9424   

   v115      v116    v117  ...    v527    v528      v571    v572   v573  \
0   0.0  748.6115  0.9908  ...  0.5064  6.6926  533.8500  2.1113   8.95   
1   0.0  731.2517  0.9902  ...  0.8832  8.8370  535.0164  2.4335   5.92   
2   0.0  718.5777  0.9899  ...  0.6451  6.4568  535.0245  2.0293  11.21   
3   0.0  709.0867  0.9906  ...  0.7404  6.4865  530.5682  2.0253   9.33   
4   0.0  796.5950  0.9908  ...  2.2181  6.3745  532.0155  2.0275   8.83   

     v574    v575    v576    v577     v578  
0  0.3157  3.0624  0.1026  1.6765  14.950

##### 헬퍼 함수 정의

In [306]:
# 오토인코더 복원오차 라인 그래프
def recon_err_plot(x, x_pred, y, threshold=0):   
    mse = np.mean(np.power(x - x_pred, 2), axis=1)
    error_df = pd.DataFrame({'Reconstruction_error': mse, 'True_class': y})
    error_df = error_df.reset_index()

    fig = px.scatter(error_df, x='index', y='Reconstruction_error', color='True_class', 
                     labels={'True_class': 'Class'}, 
                     color_continuous_scale=px.colors.sequential.Viridis,
                     title="Reconstruction error for different classes")
    fig.add_hline(y=threshold, line_color="red", annotation_text="Threshold", 
                  annotation_position="bottom right")
    fig.update_layout(xaxis_title="Data point index", yaxis_title="Reconstruction error")
    fig.show()

    return error_df


In [307]:
# 이상치점수별 각 평가지표 곡선
def prec_rec_f1_curve(y, score, pos = 1) :
    precision, recall, thresholds = precision_recall_curve(y, score, pos_label=1)
    f1 = 2 / (1/precision + 1/recall)
    thres_f1_max = thresholds[np.where(f1 == f1.max())][0]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=thresholds, y=np.delete(precision, -1), mode='lines', name='precision'))
    fig.add_trace(go.Scatter(x=thresholds, y=np.delete(recall, -1), mode='lines', name='recall'))
    fig.add_trace(go.Scatter(x=thresholds, y=np.delete(f1, -1), mode='lines', name='f1'))
    fig.add_vline(x=thres_f1_max, line_color="red", line_dash="dash", line_width=1, annotation_text="{:.2f}".format(thres_f1_max), annotation_position="bottom left", annotation_font_color="red")
    fig.add_hline(y=f1.max(), line_color="red", line_dash="dash", line_width=1)
    fig.add_annotation(x=thres_f1_max, y=f1.max(), text="최대 F1 점수: {:.2f}".format(f1.max()), showarrow=True, arrowhead=1)
    fig.update_layout(title='Precision, Recall, F1 Score vs Anomaly Score',
                      xaxis_title='Anomaly Score',
                      yaxis_title='Score',
                      legend_title='Metrics')
    fig.show()

    return precision, recall, f1, thresholds


In [308]:
# 혼동 행렬, 분류 보고서
def custom_classification_report(y, pred, thresholds):
    pred_temp = np.where(pred > thresholds , 1, 0)

    cm = confusion_matrix(y, pred_temp)
    cm_df = pd.DataFrame(cm, index=['Actual Negative', 'Actual Positive'], columns=['Predicted Negative', 'Predicted Positive'])
    print('< confusion matrix >\n')
    print('\n' + '='*60 + '\n')
    print(cm_df)
    print('\n')
    cr = classification_report(y, pred_temp, output_dict=True)
    cr_df = pd.DataFrame(cr).transpose()
    print('< classification_report >\n')
    print(cr_df)

##### 데이터 전처리

In [310]:
data['label'] = 0
data.loc[data['defeat']== 'defeat', 'label']= 1
data.drop(['datetime','defeat'], axis = 1, inplace=True)

target = 'label'
x = data.drop(target, axis = 1)
y = data.loc[:,target]

In [311]:
# 이상 데이터 비율 확인
print(y.value_counts())
print(y.value_counts() / y.shape[0])

label
0    1463
1     104
Name: count, dtype: int64
label
0    0.933631
1    0.066369
Name: count, dtype: float64


In [314]:
# 훈련, 테스트 데이터 나누기
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .2, random_state=2021)
x_train.shape, y_train.shape

((1253, 52), (1253,))

In [16]:
# 스케일링
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

In [315]:
# 학습을 위해서는 Normal 데이터만 이용한다.
x_train0 = x_train[y_train == 0]
x_train0.shape

(1166, 52)

##### 모델링

In [316]:
# 파라미터 설정
epochs = 100
batch_size = 64
input_dim = x_train0.shape[1] # 입력 레이어 차원은 필드의 수와 같다.
input_dim

52

In [321]:
clear_session()

# 입력 레이어
input_layer = Input(shape=(input_dim, ))

# 인코더 레이어 정의
encoder = Dense(32, activation="relu")(input_layer)
encoder = Dense(16, activation="relu")(encoder)

# 디코더 레이어 정의
decoder = Dense(32, activation='relu')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)

# 모델 정의
autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 52)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         1,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 52)             │         1,716 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,484 (17.52 KB)

 Trainable params: 4,484 (17.52 KB)

 Non-trainable params: 0 (0.00 B)

In [322]:
# 모델 컴파일
autoencoder.compile(optimizer='adam',loss='mse', metrics=['accuracy'])
# 모델 훈련
history = autoencoder.fit(x=x_train0, y=x_train0,
                          epochs=epochs,
                          batch_size=batch_size,
                          validation_data=(x_val, x_val)).history

Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0000e+00 - loss: 77342.3906 - val_accuracy: 0.0000e+00 - val_loss: 70681.0625
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 7.0336e-04 - loss: 67632.6719 - val_accuracy: 0.0191 - val_loss: 54050.5586
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8214 - loss: 46843.2266 - val_accuracy: 1.0000 - val_loss: 21610.2051
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9983 - loss: 15629.5908 - val_accuracy: 1.0000 - val_loss: 4223.1260
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9992 - loss: 3229.4771 - val_accuracy: 1.0000 - val_loss: 1125.5347
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9989 - loss: 1223.7025 - val_accuracy: 1.0000 - val_loss: 1007.7110
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9968 - loss: 1378.5551 - val_accuracy: 1.0000 - val_loss: 956.4745
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

In [323]:
# 훈련 결과 시각화
fig = go.Figure()
fig.add_trace(go.Scatter(y=history['loss'], mode='lines', name='Train'))
fig.add_trace(go.Scatter(y=history['val_loss'], mode='lines', name='Validation'))

fig.update_layout(title='Model loss',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')
#fig.update_yaxes(range=[0.70, 1])
fig.show()


##### 

##### 모델 평가

In [324]:
# 이상 데이터 예측
pred = autoencoder.predict(x_val)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [325]:
# 복원 오차 라인 그래프
result = recon_err_plot(x_val, pred, y_val, 0.01)

In [326]:
# 이상 점수별 각 평가 지표 곡선
precision, recall, f1, thresholds = prec_rec_f1_curve(result['True_class'], result['Reconstruction_error'])

In [328]:
# 혼동 행렬, 분류 보고서
thres_f1_max = thresholds[np.where(f1 == f1.max())][0]
custom_classification_report(result['True_class'], result['Reconstruction_error'], thres_f1_max)


< confusion matrix >



                 Predicted Negative  Predicted Positive
Actual Negative                 242                  55
Actual Positive                  13                   4


< classification_report >

              precision    recall  f1-score     support
0              0.949020  0.814815  0.876812  297.000000
1              0.067797  0.235294  0.105263   17.000000
accuracy       0.783439  0.783439  0.783439    0.783439
macro avg      0.508408  0.525054  0.491037  314.000000
weighted avg   0.901310  0.783439  0.835040  314.000000


### 2) 오토인코더 동적 모델링 (세콤 데이터 이상 탐지)

이번에는 오토인코더 클래스를 만들어서 동일한 모델링을 하고,
데이터셋을 바꿔서 진행합니다.

In [329]:
# 케라스 Model 클래스를 상속받아 오토인코더 모델을 재정의합니다.
class Autoencoder(Model):
  
  def __init__(self, input_dim, encode_layers, decode_layers): # 레이어의 깊이와 노드 수를 동적으로 받아 모델링이 가능하도록 만듭니다.
    super(Autoencoder, self).__init__()
    self.encoder_layers = []
    self.decoder_layers = []
    
    # 인코더 레이어 생성
    self.encoder_layers.append(Dense(encode_layers[0], activation="relu", input_shape=(input_dim,)))
    for layer_size in encode_layers[1:]:
      self.encoder_layers.append(Dense(layer_size, activation="relu"))
    
    # 디코더 레이어 생성
    self.decoder_layers.append(Dense(decode_layers[0], activation='relu'))
    for layer_size in decode_layers[1:]:
      self.decoder_layers.append(Dense(layer_size, activation='relu'))
    self.decoder_layers.append(Dense(input_dim, activation='relu'))
    
    # 모델 구성
    self.autoencoder_model = Sequential(self.encoder_layers + self.decoder_layers)
    self.autoencoder_model.summary()
    
  def call(self, inputs):
    return self.autoencoder_model(inputs)
    

  # 평가 그래프를 종합해서 모두 그립니다.
  def plot(self, x_val, y_val, threshold=0.01):
    self.pred = self.predict(x_val)
    mse = np.mean(np.power(x_val - self.pred, 2), axis=1)
    self.error_df = pd.DataFrame({'Reconstruction_error': mse, 'True_class': y_val}).reset_index()
    
    self.plot_recon_err(self.error_df, threshold)
    self.precision, self.recall, self.f1, self.thresholds = self.prec_rec_f1_curve(y_val, self.error_df['Reconstruction_error'])
    self.confusion_matrix, self.classification_report = self.classification_report(y_val, self.error_df['Reconstruction_error'], threshold)
  
  # 복원 에러 그래프
  @staticmethod
  def plot_recon_err(error_df, threshold):
    fig = px.scatter(error_df, x='index', y='Reconstruction_error', color='True_class', 
                     labels={'True_class': 'Class'}, 
                     color_continuous_scale=px.colors.sequential.Viridis,
                     title="Reconstruction error for different classes")
    fig.add_hline(y=threshold, line_color="red", annotation_text="Threshold", 
                  annotation_position="bottom right")
    fig.update_layout(xaxis_title="Data point index", yaxis_title="Reconstruction error")
    fig.show()
  
  # 평가지표 곡선
  @staticmethod
  def prec_rec_f1_curve(y, score) :
    precision, recall, thresholds = precision_recall_curve(y, score, pos_label=1)
    f1 = 2 / (1/precision + 1/recall)
    thres_f1_max = thresholds[np.where(f1 == f1.max())][0]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=thresholds, y=np.delete(precision, -1), mode='lines', name='precision'))
    fig.add_trace(go.Scatter(x=thresholds, y=np.delete(recall, -1), mode='lines', name='recall'))
    fig.add_trace(go.Scatter(x=thresholds, y=np.delete(f1, -1), mode='lines', name='f1'))
    fig.add_vline(x=thres_f1_max, line_color="red", line_dash="dash", line_width=1, annotation_text="{:.2f}".format(thres_f1_max), annotation_position="bottom left", annotation_font_color="red")
    fig.add_hline(y=f1.max(), line_color="red", line_dash="dash", line_width=1)
    fig.add_annotation(x=thres_f1_max, y=f1.max(), text="최대 F1 점수: {:.2f}".format(f1.max()), showarrow=True, arrowhead=1)
    fig.update_layout(title='Precision, Recall, F1 Score vs Anomaly Score',
                      xaxis_title='Anomaly Score',
                      yaxis_title='Score',
                      legend_title='Metrics')
    fig.show()

    return precision, recall, f1, thresholds
  
  # 혼동 행렬, 분류 보고서
  @staticmethod
  def classification_report(y, pred, thresholds):
    pred_temp = np.where(pred > thresholds , 1, 0)

    cm = confusion_matrix(y, pred_temp)
    cm_df = pd.DataFrame(cm, index=['Actual Negative', 'Actual Positive'], columns=['Predicted Negative', 'Predicted Positive'])
    print('< confusion matrix >\n')
    print('\n' + '='*60 + '\n')
    print(cm_df)
    print('\n')
    cr = classification_report(y, pred_temp, output_dict=True)
    cr_df = pd.DataFrame(cr).transpose()
    print('< classification_report >\n')
    print(cr_df)
    return cm_df, cr_df


##### 모델링

In [331]:
# 입력 레이어 차원
input_dim = x_train0.shape[1]
input_dim

52

In [332]:
autoencoder = Autoencoder(input_dim=input_dim, encode_layers=[32, 24, 16, 8], decode_layers=[16, 24, 32, input_dim])

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 32)             │         1,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 24)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │           400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 24)             │           408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │           800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 52)             │         1,716 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 52)             │         2,756 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,848 (34.56 KB)

 Trainable params: 8,848 (34.56 KB)

 Non-trainable params: 0 (0.00 B)

##### 모델 훈련

In [153]:
autoencoder.compile(optimizer='adam', loss=MeanSquaredError())

In [154]:
autoencoder.fit(x_train0, x_train0, epochs=epochs, batch_size=batch_size, validation_data=(x_val, x_val))

Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1226 - val_loss: 0.0828
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0760 - val_loss: 0.0636
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0612 - val_loss: 0.0477
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0430 - val_loss: 0.0386
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0375 - val_loss: 0.0378
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0365 - val_loss: 0.0375
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0372 - val_loss: 0.0371
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0369 - val_loss: 0.0363
Epoch 9/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0353 - val_loss: 0.0357
Epoch 10/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0355 - val_loss: 0.0354
Epoch 11/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0347 - val_loss: 0.0353
Epoch 12/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0

##### 모델 평가

In [155]:
autoencoder.plot(x_val, y_val, 0.01)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


< confusion matrix >



                 Predicted Negative  Predicted Positive
Actual Negative                  67                 230
Actual Positive                   3                  14


< classification_report >

              precision    recall  f1-score     support
0              0.957143  0.225589  0.365123  297.000000
1              0.057377  0.823529  0.107280   17.000000
accuracy       0.257962  0.257962  0.257962    0.257962
macro avg      0.507260  0.524559  0.236201  314.000000
weighted avg   0.908429  0.257962  0.351163  314.000000


### 3) 오토인코더 모델링의 비즈니스 평가 (종이공장 데이터 장애 탐지)

* 참고 이미지

    <img src="https://i.pinimg.com/originals/3c/cd/c7/3ccdc785f2388f6e0ef9fc0ff561c7bc.jpg" width="500">

#### 가상 프로젝트 시나리오
- 비즈니스 배경
   - 펄프 및 제지 공장 데이터셋은 제지 산업의 생산 공정에서 발생하는 다양한 데이터를 포함하고 있음. 이러한 데이터셋은 제지 공장의 기계와 공정에서 수집된 센서 데이터, 운영 로그, 품질 측정 결과 등을 포함할 수 있음. 이 데이터를 활용하여 공정의 이상 징후를 탐지하고, 생산성을 높이며, 유지보수 비용을 절감하는 등의 작업을 수행할 수 있음.
   <br/>
- 구체적 비즈니스 상황
   - 한 롤로 종이를 말다가 찢어지는 사고가 하루에 한번 씩 발생함.
   - 이때마다 공정 중단 및 수율 저하 등의 이유로 평균 100만원 손실 발생.
   - 사전에 감지하는 것이 중요하지만 이를 예상하기는 어려움.
   - 5~10%만 감소 시킬수 있다면 상당한 비용절감 효과가 예상됨.
 <br/>   
- 본 프로젝트로 인한 기타 효익
   - 생산 효율성 증가: 이상 징후를 조기에 감지하여 공정 중단을 최소화하고, 생산성을 높임.
   - 품질 관리: 제품의 품질을 일관되게 유지하며, 품질 문제가 생산 초기에 발견되도록 함.
   - 비용 절감: 고장이나 비효율적인 운영으로 인한 비용을 줄임. 유지보수 작업을 계획적으로 수행하여 긴급 수리 비용과 다운타임을 줄일 수 있음.

- 데이터셋의 주요 구성 요소 (데이터셋의 필드는 마스킹 처리됨)
   - 센서 데이터: 기계의 온도, 압력, 속도, 전력 사용량 등을 실시간으로 모니터링하는 센서에서 수집된 데이터. 이러한 데이터는 기계의 정상 작동 범위를 벗어난 활동을 감지하는 데 중요함.
   - 운영 로그: 기계의 작동 시간, 고장 및 유지보수 기록, 작업자의 교대 시간표 등이 포함됨. 이 정보는 기계의 성능과 고장 패턴 분석에 유용함.
   - 품질 데이터: 완성된 제품의 두께, 무게, 강도 등 품질 관련 측정치. 제품 품질이 사양을 벗어났을 경우 이상 상태로 판단할 수 있음.
   - 환경 데이터: 작업 환경의 온도, 습도와 같은 조건을 기록한 데이터. 이는 생산 공정에 영향을 미칠 수 있는 중요한 변수임.
<br/>
- 데이터 구조
   - 15일간 18000건의 트랙잭션 시계열 데이터
   - 시간 간격은 2분
   - y : 0은 정상, 1은 비정상 데이터
   - x1 ~ x61 : 센서, 운영, 품질, 환경 데이터
<br/>
- 주요 고려사항
   - y가 1인 데이터는 장애 발생 시점이기 때문에, 사전예측을 위해서는 2-4분 전 예측이 가능하도록 데이터 Shift가 필요
   - t(n) 이 1(장애) 라면 t(n-2)와 t(n-1) 로 1로 값을 치환해주고, t(n)인 데이터는 행삭제 (장배 발생 시점 데이터를 이용해 다음을 예측하는 것은 이치에 맞지 않기 때문)
   - 비즈니스 제한사항 : 장애가 예측되면 속도를 출여 장애를 예방(100만원 손실 예방) 가능하지만, 생산성 저하로 속도 감소 1회 진행시 5만원 손실 발생

##### 데이터 로드

In [160]:
# 공정 데이터 불러오기
data = pd.read_csv('dataset/process_data.csv')
data.head()

,time,y,x1,x2,x3,x4,x5,x6,x7,x8,...,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60
0,5/1/99 0:00,0,0.376665,-4.596435,-4.095756,13.497687,-0.118830,-20.669883,0.000732,-0.061114,...,29.984624,10.091721,0.053279,-4.936434,-24.590146,18.515436,3.473400,0.033444,0.953219,0.006076
1,5/1/99 0:02,0,0.475720,-4.542502,-4.018359,16.230659,-0.128733,-18.758079,0.000732,-0.061114,...,29.984624,10.095871,0.062801,-4.937179,-32.413266,22.760065,2.682933,0.033536,1.090502,0.006083
2,5/1/99 0:04,0,0.363848,-4.681394,-4.353147,14.127997,-0.138636,-17.836632,0.010803,-0.061114,...,29.984624,10.100265,0.072322,-4.937924,-34.183774,27.004663,3.537487,0.033629,1.840540,0.006090
3,5/1/99 0:06,0,0.301590,-4.758934,-4.023612,13.161566,-0.148142,-18.517601,0.002075,-0.061114,...,29.984624,10.104660,0.081600,-4.938669,-35.954281,21.672449,3.986095,0.033721,2.554880,0.006097
4,5/1/99 0:08,0,0.265578,-4.749928,-4.333150,15.267340,-0.155314,-17.505913,0.000732,-0.061114,...,29.984624,10.109054,0.091121,-4.939414,-37.724789,21.907251,3.601573,0.033777,1.410494,0.006105


##### 이상치 분포 확인

In [207]:
date_tmp = data.copy()
date_tmp['time'] = pd.to_datetime(data.time, format = '%m/%d/%y %H:%M')
date_tmp['day'] = date_tmp['time'].dt.day
date_tmp.loc[date_tmp['y']==1].groupby('day', as_index=False)['y'].count()

,day,y
0,1,1
1,2,6
2,3,3
3,4,11
4,5,11
5,6,4
6,7,2
7,8,2
8,9,1
9,10,1


##### 데이터 전처리 (중요)

In [209]:
new_data = data.copy()
new_data['new_y'] = 0
for i, row in new_data.iterrows():
    if row['y'] == 1:
        if i > 1:
            new_data['new_y'][i-2] = 1
        if i > 0:
            new_data['new_y'][i-1] = 1
new_data = new_data[new_data['y'] != 1]
new_data.drop(['y'], axis=1, inplace=True)
new_data.rename(columns={'new_y': 'y'}, inplace=True)

In [211]:
new_data.drop('time', axis=1, inplace=True)

In [212]:
new_data.shape

(18274, 60)

In [213]:
train, test = train_test_split(new_data, test_size=4000)
train, val = train_test_split(train, test_size=4000)

In [214]:
x_train_0 = train.loc[train['y'] == 0].drop(['y'], axis=1)
x_val_0 = val.loc[val['y'] == 0].drop(['y'], axis=1)

In [215]:
# 스케일링 fitting
scaler = StandardScaler()

# 적용
x_train_0_s = scaler.fit_transform(x_train_0)
x_val_0_s = scaler.transform(x_val_0) #학습시 validation용

x_test_s = scaler.transform(test.drop(['y'], axis = 1)) # 모델 평가용.

#### 모델링

In [260]:
input_dim = x_train_0_s.shape[1] 
autoencoder = Autoencoder(input_dim=input_dim, encode_layers=[32, 24, 16, 8], decode_layers=[16, 24, 32, input_dim])

Model: "sequential_37"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_189 (Dense)               │ (None, 32)             │         1,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_190 (Dense)               │ (None, 24)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_191 (Dense)               │ (None, 16)             │           400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_192 (Dense)               │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_193 (Dense)               │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_194 (Dense)               │ (None, 24)             │           408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_195 (Dense)               │ (None, 32)             │           800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_196 (Dense)               │ (None, 59)             │         1,947 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_197 (Dense)               │ (None, 59)             │         3,540 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,087 (39.40 KB)

 Trainable params: 10,087 (39.40 KB)

 Non-trainable params: 0 (0.00 B)

In [261]:
from keras.optimizers import Adam
autoencoder.compile(optimizer= Adam(learning_rate=1e-3), loss='mse', metrics=['accuracy'])
autoencoder.fit(x_train_0_s, x_train_0_s, epochs=200, batch_size=batch_size, validation_data=(x_val_0_s, x_val_0_s))

Epoch 1/200
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0300 - loss: 0.9629 - val_accuracy: 0.0881 - val_loss: 0.8092
Epoch 2/200
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1235 - loss: 0.8309 - val_accuracy: 0.1649 - val_loss: 0.7650
Epoch 3/200
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1752 - loss: 0.7766 - val_accuracy: 0.1836 - val_loss: 0.7436
Epoch 4/200
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 912us/step - accuracy: 0.1915 - loss: 0.7785 - val_accuracy: 0.2328 - val_loss: 0.7329
Epoch 5/200
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 999us/step - accuracy: 0.2257 - loss: 0.7598 - val_accuracy: 0.2515 - val_loss: 0.7273
Epoch 6/200
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2416 - loss: 0.7700 - val_accuracy: 0.2642 - val_loss: 0.7211
Epoch 7/200
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 972us/step - accuracy: 0.2635 - loss: 0.7655 - val_accuracy: 0.2751 - val_loss: 0.7170
Epoch 8/200
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2852 - loss: 0.7227 - va

##### 모델평가 by 평가지표

In [262]:
autoencoder.plot(x_test_s, test['y'], 1)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 476us/step


< confusion matrix >



                 Predicted Negative  Predicted Positive
Actual Negative                3712                 244
Actual Positive                  42                   2


< classification_report >

              precision    recall  f1-score    support
0              0.988812  0.938322  0.962905  3956.0000
1              0.008130  0.045455  0.013793    44.0000
accuracy       0.928500  0.928500  0.928500     0.9285
macro avg      0.498471  0.491888  0.488349  4000.0000
weighted avg   0.978024  0.928500  0.952465  4000.0000


##### 모델평가 by 비즈니스 관점

|              | 정상 예측(0) | 장애 예측(1) |
|--------------|-------------|-------------|
| 정상 작동(0) | TN_cost = 0  | FP_cost = 5 |
| 장애 발생(1) | FN_cost = 100| TP_cost = 5 |

In [295]:
# 기대 손실비용 변수
TN_cost, FP_cost, FN_cost, TP_cost = 0, 5, 100, 5

In [296]:
autoencoder.thresholds

array([ 0.12378361,  0.14743062,  0.15880029, ..., 14.9587053 ,
       15.03159872, 15.10817549])

In [297]:
autoencoder.error_df.head(10)

,index,Reconstruction_error,True_class
0,14769,0.620562,0
1,7011,0.186665,0
2,7400,0.231928,0
3,15202,0.604609,0
4,12660,0.534241,0
5,16544,14.637194,0
6,6599,0.195760,0
7,4376,0.912057,0
8,15063,0.957681,0
9,10559,0.898837,0


In [300]:
cost = []
for at in autoencoder.thresholds :
    y_pred = np.where(autoencoder.error_df['Reconstruction_error'] > at, 1, 0)
    TN, FP, FN, TP = confusion_matrix(autoencoder.error_df['True_class'], y_pred).ravel()
    cost.append(TP*TP_cost + FP*FP_cost + FN*FN_cost + TN*TN_cost)

cost = np.array(cost)
base_cost = autoencoder.error_df['True_class'].sum() * 100 # 실제 발생한 손실

In [301]:
fig = px.line(x=autoencoder.thresholds, y=cost, title='Cost vs Thresholds', labels={'x': 'Thresholds', 'y': 'Cost'})
fig.add_hline(y=base_cost, line_color='red', line_width=2, line_dash='dash', annotation_text="Base Cost")
fig.show()

In [294]:
best_threshold = autoencoder.thresholds[list(cost).index(cost.min())]
best_threshold


15.108175491681799